In [11]:
import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 0) #2 titik perpuluhan


date = '20211231'
#jgn lupa rename nme file


date1 = "Dec'21"
GB = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\01_Corporate_Financing\01_Portfolio\\'+date+'\working - SH\GB Monthly Loan Outstanding '+date1+'.xlsx', sheet_name = 'SourceCM') 
#sheet_name='cb os by mkt seg 2'
#sheet_name='SourceCM'


description = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\Python\User defined\description.xlsx", sheet_name='description')


Report_Corp_CRDM = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\11_CRRS\\'+date[:4]+'\\'+date+'\Report_Corp_CRDM_'+date+'.xlsx')


#x install pyxlsb so xleh import 2007 punya excel, engine='pyxlsb'
date2 = 'Dec 2021'
CRWA = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\12_RDMS\\'+date+'\working - SH\CRWA '+date2+' Final (Domestic - All except Retail).xlsx', sheet_name='Context Id=162')
#sheet_name='Context Id=139'
#sheet_name='Context Id=162'


List = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\12_RDMS\\'+date+'\working - SH\List Master Listing '+date+'.xlsx')
#, sheet_name='Sheet2'

In [12]:
GB.columns = GB.columns.str.strip()
GB.columns = GB.columns.str.upper()
GB.columns = GB.columns.str.replace(" ","_")

GB.M_CUS_NO = GB.M_CUS_NO.astype(str)

#case 20220831
#GB['M_CUS_NO'] = GB['M_CUS_NO'].astype('Int64').astype('str')

GB.M_FINANCING_TYPE_CD = GB.M_FINANCING_TYPE_CD.str.strip()
GB.M_FINANCING_TYPE_CD = GB.M_FINANCING_TYPE_CD.str.upper()

GB.FACILITY = GB.FACILITY.str.strip()
GB.FACILITY = GB.FACILITY.str.upper()

#case 20211231,20220331,20210930 nme M_BNM_BALANCE_SUM_1
#GB['M_BNM_BALANCE_SUM']=GB['M_BNM_BALANCE_SUM_1']

#M_BNM_BALANCE_SUM 20220731
GB.M_BNM_BALANCE_SUM.fillna(0, inplace=True)

GB1 = GB.iloc[np.where((GB.M_FINANCING_TYPE_CD=='I')&(GB.FACILITY!='RTO'))]


#case 20211231
#check M_MSIC_CODE kat excel n let type as general sbb klau x dia jd time value

GB1=GB1[['M_CUS_NO','NAME','M_MSIC_CODE','M_BNM_BALANCE_SUM']]

GB1 = GB1.fillna(0).groupby(['M_CUS_NO','NAME','M_MSIC_CODE'])[['M_BNM_BALANCE_SUM']].sum().reset_index().sort_values(by='M_BNM_BALANCE_SUM', ascending=False)


description.columns = description.columns.str.strip()
description.columns = description.columns.str.replace(" ", "_")
description.columns = description.columns.str.upper()

description.ISC_NOB = description.ISC_NOB.map(str)
description.ISC_NOB = description.ISC_NOB.str.strip()
description.ISC_NOB = description.ISC_NOB.str.upper()

GB1.M_MSIC_CODE = GB1.M_MSIC_CODE.map(str)
GB1.M_MSIC_CODE = GB1.M_MSIC_CODE.str.strip()
GB1.M_MSIC_CODE = GB1.M_MSIC_CODE.str.upper()

join_A = GB1.merge(description[['ISC_NOB','INDUSTRIAL_SECTOR_CODE_(ISC_DESC)2','ISC_PARENT']].drop_duplicates().rename(columns = {'ISC_NOB':'M_MSIC_CODE'}), on='M_MSIC_CODE', how='left').drop('M_BNM_BALANCE_SUM', axis=1)


Report_Corp_CRDM.columns = Report_Corp_CRDM.columns.str.strip()
Report_Corp_CRDM.columns = Report_Corp_CRDM.columns.str.upper()
Report_Corp_CRDM.columns = Report_Corp_CRDM.columns.str.replace(" ","_")

Report_Corp_CRDM1 = Report_Corp_CRDM[['GCIF_NUM','RDMS_ADJ_BRR']]

Report_Corp_CRDM1.GCIF_NUM = Report_Corp_CRDM1.GCIF_NUM.map(str)
Report_Corp_CRDM1.GCIF_NUM = Report_Corp_CRDM1.GCIF_NUM.str.strip()
Report_Corp_CRDM1.GCIF_NUM = Report_Corp_CRDM1.GCIF_NUM.str.upper()

join_A.M_CUS_NO = join_A.M_CUS_NO.map(str)
join_A.M_CUS_NO = join_A.M_CUS_NO.str.strip()
join_A.M_CUS_NO = join_A.M_CUS_NO.str.upper()

#case 20210630
#join_A['M_CUS_NO'] = join_A['M_CUS_NO'].str[:-2]

join_B = join_A.merge(Report_Corp_CRDM1.rename(columns = {'GCIF_NUM':'M_CUS_NO'}).drop_duplicates(), on='M_CUS_NO', how='left')




CRWA.columns = CRWA.columns.str.strip()
CRWA.columns = CRWA.columns.str.upper()
CRWA.columns = CRWA.columns.str.replace(" ","_")

CRWA1 = CRWA[['COUNTERPARTY_ORIGINAL','CUSTOMER_FIELD19','INTERNAL_RATING_ORIGINAL']]

CRWA1.COUNTERPARTY_ORIGINAL = CRWA1.COUNTERPARTY_ORIGINAL.map(str)
CRWA1.COUNTERPARTY_ORIGINAL = CRWA1.COUNTERPARTY_ORIGINAL.str.strip()
CRWA1.COUNTERPARTY_ORIGINAL = CRWA1.COUNTERPARTY_ORIGINAL.str.upper()

join_C = join_B.merge(CRWA1.rename(columns = {'COUNTERPARTY_ORIGINAL':'M_CUS_NO'}).drop_duplicates('M_CUS_NO'), on='M_CUS_NO', how='left').drop_duplicates()


def PSE_SOV(CUSTOMER_FIELD19):
    if CUSTOMER_FIELD19 == 'PSE(SA)':
        return "PSE"
    elif CUSTOMER_FIELD19 == 'SOV(SA)':
        return "SOV"
#    else:
#        return "Unrated"
join_C['PSE/SOV'] = join_C.CUSTOMER_FIELD19.apply(PSE_SOV)

List.columns = List.columns.str.strip()
List.columns = List.columns.str.upper()
List.columns = List.columns.str.replace(" ","_")

List.GCIF_CHILD = List.GCIF_CHILD.map(str)
List.GCIF_CHILD = List.GCIF_CHILD.str.strip()
List.GCIF_CHILD = List.GCIF_CHILD.str.upper()

join_D = join_C.merge(List.rename(columns={"GCIF_CHILD":"M_CUS_NO"}).drop_duplicates(), on='M_CUS_NO', how='left').drop_duplicates()


#join_D.GCIF_PARENTS = join_D.GCIF_PARENTS.apply(lambda x: f'{x:.0f}')
join_D.GCIF_PARENTS = join_D.GCIF_PARENTS.astype(str)
join_D.GCIF_PARENTS = join_D.GCIF_PARENTS.str.strip()
join_D.GCIF_PARENTS = join_D.GCIF_PARENTS.str.upper()

join_D['GCIF_PARENTS1'] = join_D.GCIF_PARENTS.str[:-2]


join_E = join_D.merge(join_D[['M_CUS_NO','INTERNAL_RATING_ORIGINAL']].rename(columns={'M_CUS_NO':'GCIF_PARENTS1','INTERNAL_RATING_ORIGINAL':'3RD_PARTY'}), on='GCIF_PARENTS1', how='left')


join_E1 = join_E.merge(CRWA1.rename(columns = {'COUNTERPARTY_ORIGINAL':'GCIF_PARENTS1'}).drop_duplicates('GCIF_PARENTS1'), on='GCIF_PARENTS1', how='left',suffixes=('', '_y')).drop_duplicates()

join_E1['CUSTOMER_FIELD19'] = join_E1.CUSTOMER_FIELD19
join_E1.loc[join_E1['CUSTOMER_FIELD19'].isnull(),'CUSTOMER_FIELD19'] = join_E1.CUSTOMER_FIELD19_y


join_E1['3RD_PARTY'] = join_E1['3RD_PARTY']
join_E1.loc[join_E1['3RD_PARTY'].isnull(),'3RD_PARTY'] = join_E1.INTERNAL_RATING_ORIGINAL_y


join_E2 = join_E1.merge(Report_Corp_CRDM1.rename(columns = {'GCIF_NUM':'GCIF_PARENTS1'}).drop_duplicates('GCIF_PARENTS1'), on='GCIF_PARENTS1', how='left',suffixes=('', '_y')).drop_duplicates()

join_E2['3RD_PARTY'] = join_E2['3RD_PARTY']
join_E2.loc[join_E2['3RD_PARTY'].isnull(),'3RD_PARTY'] = join_E2.RDMS_ADJ_BRR_y


join_E2['FINAL_RATING'] = join_E2['INTERNAL_RATING_ORIGINAL']
join_E2.loc[join_E2['FINAL_RATING'].isnull(),'FINAL_RATING'] = join_E2['3RD_PARTY']
join_E2.loc[join_E2['FINAL_RATING'].isnull(),'FINAL_RATING'] = join_E2.RDMS_ADJ_BRR
join_E2.loc[join_E2['FINAL_RATING'].isnull(),'FINAL_RATING'] = join_E2['PSE/SOV']
join_E2.loc[join_E2['FINAL_RATING'].isnull(),'FINAL_RATING'] = "Unrated"


join_E2.rename(columns={'M_MSIC_CODE':'BNM_SECTOR_CD',\
                       'INDUSTRIAL_SECTOR_CODE_(ISC_DESC)2':'SECTOR_DESC',\
                      'ISC_PARENT':'SECTOR_PARENT',\
                      'RDMS_ADJ_BRR':'CRRS',\
                      'INTERNAL_RATING_ORIGINAL':'RDMS',\
                      'GCIF_PARENTS1':'3RD_PARTY_PARENT'},inplace=True)

join_E2['SEGMENT'] = '01. CB'

join_E2 = join_E2[['M_CUS_NO',\
                'NAME',\
                'BNM_SECTOR_CD',\
                'SECTOR_DESC',\
                'SECTOR_PARENT',\
                'SEGMENT',\
                'RDMS',\
                'CUSTOMER_FIELD19',\
                'PSE/SOV',\
                'CRRS',\
                '3RD_PARTY_PARENT',\
                '3RD_PARTY',\
                'FINAL_RATING']]

D:\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [15]:
writer2 = pd.ExcelWriter(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\18_Rating_CBBBRSME\BRR_CBBBRSME_'+date[:6]+'(python-new rules).xlsx',engine='xlsxwriter')

join_E2.to_excel(writer2, index = False)

writer2.save()

In [ ]:
=='Unrated' !='N'

In [16]:
a = join_E2.iloc[np.where(join_E2['FINAL_RATING']=='Unrated')]
a.shape

(49, 13)

# FIX SECTION

In [13]:
Nisa = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\18_Rating_CBBBRSME\BRR_CBBBRSME_202208.xlsx', sheet_name='Sheet1', header=0)

Nisa.columns = Nisa.columns.str.replace(" ","_").str.upper()
Nisa1 = Nisa.iloc[np.where(Nisa.SEGMENT=='01. CB')][['M_CUS_NO','FINAL_RATING']]

In [20]:
Nisa['M_CUS_NO'] = Nisa['M_CUS_NO'].astype('Int64').astype('str')


In [55]:
CEK = join_E2.merge(Nisa[['M_CUS_NO','FINAL_RATING']],on='M_CUS_NO',how='left',indicator=True, suffixes=('','_y'))

In [56]:
Nisa1.shape, join_E.shape, CEK.shape

((551, 2), (518, 12), (518, 15))

In [65]:
#CEK._merge.value_counts()

In [64]:
y = CEK.iloc[np.where(CEK.FINAL_RATING_y!='Unrated')]
#y.iloc[np.where(y.FINAL_RATING=='Unrated')]

In [63]:
#CEK.iloc[np.where(CEK.M_CUS_NO.isin(['7051644','2182543','27777772','27777242']))]